In [ ]:
rsync -avzhe ssh \
conchae@garnatxa.srv.cpd:/home/conchae/prediction_depolymerase_tropism/prophage_prediction/depolymerase_decipher/ficheros_28032023/train_nn/fine_tuning/log_files_0712 \
/media/concha-eloko/Linux/PPT_clean/in_vitro/fine_tuning

rsync -avzhe ssh \
conchae@garnatxa.srv.cpd:/home/conchae/prediction_depolymerase_tropism/prophage_prediction/depolymerase_decipher/ficheros_28032023/train_nn/fine_tuning/models_1401 \
/media/concha-eloko/Linux/PPT_clean/in_vitro/fine_tuning

rsync -avzhe ssh \
conchae@garnatxa.srv.cpd:/home/conchae/prediction_depolymerase_tropism/prophage_prediction/depolymerase_decipher/ficheros_28032023/train_nn/fine_tuning/log_files_1401 \
/media/concha-eloko/Linux/PPT_clean/in_vitro/fine_tuning




> Get the models at the right place : 

model     LR     weight decay
model     0.0001     0.000001
model_01  1e-6       0.001

> Make dico epochs :

In [ ]:
import os 
import pandas as pd

path_log = "/media/concha-eloko/Linux/PPT_clean/in_vitro/fine_tuning/log_files_1401"

with open(f"/media/concha-eloko/Linux/PPT_clean/in_vitro/fine_tuning/epoch_file.tsv", "w") as outfile : 
    outfile.write("KL_type\tEpoch\n")
    for log_file in os.listdir(path_log) :
        try : 
            kltype = log_file.split("__")[0]
            eg_df = pd.read_csv(f"{path_log}/{log_file}", sep = "\t", names = ["epoch", "train_loss"])
            eg_df["Epoch"] = eg_df["epoch"].apply(lambda x : x.split(": ")[1])
            eg_df["Train_Loss"] = eg_df["train_loss"].apply(lambda x : float(x.split(": ")[1]))
            min_value_index = eg_df['Train_Loss'].idxmin()
            row_with_min_value = eg_df.loc[min_value_index]
            e = row_with_min_value["Epoch"]    
            outfile.write(f"{kltype}\t{e}\n")
        except IndexError :
            pass


> Open the adequate finetuned model :

In [ ]:
import os 

path_finetuning = "/media/concha-eloko/Linux/PPT_clean/in_vitro/fine_tuning/"
path_finetuned_basic = f"{path_finetuning}/models_1401"

path_new_ensemble = "/media/concha-eloko/Linux/PPT_clean/ficheros_28032023/ensemble_1401_finetuned"
path_base = "/media/concha-eloko/Linux/PPT_clean/ficheros_28032023/ensemble_2812"

df_epochs = pd.read_csv(f"{path_finetuning}/epoch_file.fixed.tsv", sep = "\t", header = 0)
df_epochs


In [ ]:
path_new_ensemble = "/media/concha-eloko/Linux/PPT_clean/ficheros_28032023/ensemble_1901_finetuned"
path_base = "/media/concha-eloko/Linux/PPT_clean/ficheros_28032023/ensemble_2812"
path_finetuned_basic = "/media/concha-eloko/Linux/PPT_clean/in_vitro/fine_tuning/models_1401"

for _, row in df_epochs.iterrows() :
    model = f"{row['KL_type']}.{row['Epoch']}.finetuned.TropiGATv2.1401.pt"
    move_model_cmd = f"cp {path_finetuned_basic}/{model} {path_new_ensemble}"
    os.system(move_model_cmd)

# Move the rest :
finetuned_kltypes = [model.split(".")[0] for model in os.listdir(path_new_ensemble)]
for model in os.listdir(path_base) : 
    if model.split(".")[0] not in finetuned_kltypes : 
        os.system(f"cp {path_base}/{model} {path_new_ensemble}")

> Clean the rep

In [ ]:
import os 

path_new_ensemble = "/media/concha-eloko/Linux/PPT_clean/ficheros_28032023/ensemble_1901_finetuned"

try :
    os.system(f"rm -rf {path_new_ensemble}")
    os.mkdir(f"{path_new_ensemble}")
except Exception as e :
    print(e)

***

In [ ]:
# Torch geometric modules
from torch_geometric.data import HeteroData, DataLoader
import torch_geometric.transforms as T
from torch_geometric.nn import to_hetero , HeteroConv , GATv2Conv
from torch_geometric.utils import negative_sampling
from torch_geometric.loader import LinkNeighborLoader

# Torch modules
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau

# SKlearn modules
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import LabelEncoder , label_binarize , OneHotEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score , matthews_corrcoef

# Ground modules
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
from itertools import product
import random
from collections import Counter
import warnings
import logging
from multiprocessing.pool import ThreadPool

# TropiGAT modules
import TropiGAT_graph
import TropiGAT_models
import TropiGAT_functions

warnings.filterwarnings("ignore")

In [ ]:
path_new_ensemble = "/media/concha-eloko/Linux/PPT_clean/ficheros_28032023/ensemble_1901_finetuned"

dico_tailored = {}
dico_models = TropiGAT_functions.make_ensemble_tailored_TropiGAT(path_new_ensemble, dico_tailored)



In [ ]:
dico_models

> Others 

In [ ]:
import pandas as pd 
import os 

path_project = "/media/concha-eloko/Linux/PPT_clean/in_vitro"

others_embeddings = pd.read_csv(f"{path_project}/Others_all.esm2.embedding.csv", sep = "," , header = None)
others_embeddings.set_index([0], inplace = True)

In [ ]:
# Run the predictions Others :
other_predictions = {}
for index, dpo in enumerate(others_embeddings.index) :
    if dpo not in ["MN781108.1_prot_QGZ15323.1_262"] :
        try : 
            graph_dpo = TropiGAT_functions.make_query_graph([others_embeddings.loc[dpo].values])
            pred = TropiGAT_functions.run_prediction(graph_dpo,dico_models)
            other_predictions[dpo] = pred
        except Exception as e :
            print(e, dpo)

In [ ]:
# format the results : 
others_pred_formated = TropiGAT_functions.format_predictions(other_predictions , sep = "_prot_")
TropiGAT_functions.clean_print(other_predictions)

In [ ]:
predictions = [other_predictions]

with open(f"/media/concha-eloko/Linux/PPT_clean/Other_pred/TropiGAT.Others.all.finetuned.optimum.1901.results.tsv", "w") as outfile:
    for prediction in predictions:
        prediction_sorted = dict(sorted(prediction.items()))
        for prot in prediction_sorted:
            if prediction_sorted[prot] == "No hits" or len(prediction_sorted[prot]) == 0:
                outfile.write(f"{prot}\tNo hits\n")
            else:
                outfile.write(f"{prot}\t")
                hits = [f"{kltype}:{round(score, 3)}" for kltype, score in prediction_sorted[prot].items()]
                sorted_hits = " ; ".join(sorted(hits, key=lambda x: float(x.split(":")[1]), reverse=True))
                outfile.write(sorted_hits)
                outfile.write("\n")

In [ ]:
path_results = f"/media/concha-eloko/Linux/PPT_clean/Other_pred/TropiGAT.Others.all.finetuned.optimum.1901.results.tsv"
results_df = pd.read_csv(path_results, sep = "\t", names = ["protein","predictions"])
df_sorted = results_df.sort_values(by='protein', ascending=True)

df_sorted.to_csv(f"/media/concha-eloko/Linux/PPT_clean/Other_pred/TropiGAT.Others.all.finetuned.optimum.ordered.1901.results.tsv", sep = "\t", header = 0, index = False)



***
# Get the metrics of Others : 

In [5]:
import os 
import pandas as pd 

path_others_df = '/media/concha-eloko/Linux/PPT_clean/Other_pred/predictions.all.others.2001.tsv'
df_metric_others = pd.read_csv(path_others_df, header = 0, sep = "\t")

In [31]:
tropiseq_score = []
tropigat_score = []
tropigat_opt_score = []

def get_hits(set_a, set_b) : 
    if len(set_a) > 0 : 
        common = set_a & set_b
        if len(common) > 0 :
            return list(common)
        else : 
            return [0]
    else :
        return [0]


for _,row in df_metric_others.iterrows() :
    targets = set([kltype.split(":")[0].strip() for kltype in row["Target KL type "].split(";")])
    # tropiseq :
    tropiseq = set([kltype.split(":")[0].strip() for kltype in row["TropiSeq predictions"].split(";")])
    tropiseq_score.append(get_hits(tropiseq, targets))
    # tropigat :
    tropigat = set([kltype.split(":")[0].strip() for kltype in row["TropiGATv2 predictions"].split(";")][0:15])
    print(tropigat)
    tropigat_score.append(get_hits(tropigat, targets))
    # tropigat finetuned :
    tropigat_opt = set([kltype.split(":")[0].strip() for kltype in row["TropiGATv2 optimum finetuned predictions "].split(";")])
    tropigat_opt_score.append(get_hits(tropigat_opt, targets))

 

{'KL45', 'KL14', 'KL15', 'KL111', 'KL128', 'KL12', 'KL24', 'KL81', 'KL102', 'KL23', 'KL27', 'KL123', 'KL30'}
{'KL22', 'KL21', 'KL14', 'KL128', 'KL27', 'KL145', 'KL29', 'KL43', 'KL62', 'KL70', 'KL52', 'KL118'}
{'KL57', 'KL46', 'KL8', 'KL128', 'KL12', 'KL38', 'KL151', 'KL27', 'KL145', 'KL60', 'KL29', 'KL123', 'KL74', 'KL118'}
{'KL21', 'KL17', 'KL46', 'KL128', 'KL111', 'KL81', 'KL118', 'KL149', 'KL62', 'KL70', 'KL74', 'KL29'}
{'KL19', 'KL45', 'KL14', 'KL2', 'KL46', 'KL128', 'KL81', 'KL151', 'KL30'}
{'KL19', 'KL15', 'KL14', 'KL45', 'KL2', 'KL46', 'KL112', 'KL128', 'KL127', 'KL48', 'KL81', 'KL108', 'KL52', 'KL74', 'KL118'}
{'KL28', 'KL19', 'KL45', 'KL14', 'KL46', 'KL128', 'KL111', 'KL8', 'KL48', 'KL24', 'KL43', 'KL22', 'KL7', 'KL29', 'KL30'}
{'KL28', 'KL19', 'KL36', 'KL15', 'KL14', 'KL45', 'KL46', 'KL128', 'KL8', 'KL48', 'KL81', 'KL60', 'KL74', 'KL118', 'KL23'}
{'KL169', 'KL45', 'KL14', 'KL64', 'KL15', 'KL46', 'KL128', 'KL81', 'KL29', 'KL22', 'KL52', 'KL47', 'KL118'}
{'KL19', 'KL15', 'KL64'

In [32]:
tuple(zip(tropiseq_score,tropigat_score, tropigat_opt_score))

(([0], [0], [0]),
 ([0], [0], [0]),
 ([0], [0], [0]),
 ([0], [0], [0]),
 ([0], [0], [0]),
 ([0], [0], [0]),
 (['KL25'], [0], [0]),
 (['KL35'], [0], [0]),
 ([0], [0], [0]),
 (['KL64'], ['KL64'], ['KL64']),
 ([0], [0], [0]),
 ([0], [0], [0]),
 ([0], [0], [0]),
 ([0], [0], [0]),
 (['KL47'], ['KL47'], ['KL47']),
 ([0], [0], [0]),
 ([0], [0], [0]),
 ([0], [0], [0]),
 ([0], [0], [0]),
 ([0], [0], [0]),
 ([0], [0], [0]),
 ([0], [0], [0]),
 ([0], ['KL64'], ['KL64']),
 (['KL64'], ['KL64'], ['KL64']),
 (['KL64'], ['KL64'], ['KL64']),
 ([0], [0], [0]),
 ([0], [0], [0]),
 (['KL47'], ['KL47'], ['KL47']),
 ([0], [0], [0]),
 ([0], [0], [0]),
 ([0], [0], [0]),
 ([0], ['KL24'], [0]),
 ([0], [0], [0]),
 ([0], [0], [0]),
 ([0], [0], [0]),
 (['KL24'], ['KL24'], ['KL24']),
 (['KL24'], ['KL24'], ['KL24']),
 ([0], [0], [0]),
 ([0], [0], [0]),
 ([0], [0], [0]),
 ([0], [0], [0]),
 ([0], [0], [0]),
 (['KL63'], [0], [0]),
 ([0], [0], [0]),
 ([0], [0], [0]),
 (['KL63'], [0], [0]),
 ([0], [0], [0]),
 ([0], [0], [0

In [29]:
tropigat_score

[[0],
 [0],
 [0],
 [0],
 [0],
 [0],
 ['KL25'],
 [0],
 [0],
 ['KL64'],
 [0],
 [0],
 [0],
 [0],
 ['KL47'],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 ['KL64'],
 ['KL64'],
 ['KL64'],
 [0],
 [0],
 ['KL47'],
 [0],
 [0],
 [0],
 ['KL24'],
 [0],
 [0],
 [0],
 ['KL24'],
 ['KL24'],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 ['KL24'],
 ['KL24']]